In [1]:
# Dependencies and Setup
import csv
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import time



In [2]:
dow_jones_df= pd.read_csv('/Users/dianeshomefolder/Desktop/Dow Jones/dow_jones2.csv')
#dow_jones_df.set_index('timestamp')
dow_jones_df.head()


,timestamp,open,high,low,close,volume
0,2019-09-27,26987.2598,27012.5391,26715.8203,26820.2500,217780000
1,2019-09-26,27004.1094,27015.0703,26803.8398,26891.1191,229180000
2,2019-09-25,26866.7109,27016.5605,26755.8594,26970.7109,237220000
3,2019-09-24,27034.0703,27079.6797,26704.9609,26807.7695,301750000
4,2019-09-23,26851.4492,27011.0703,26831.3398,26949.9902,204240000


In [3]:
# Create a new column ('close-minus-open') and perform the calculation
dow_jones_close = dow_jones_df
dow_jones_close['close-open'] = dow_jones_df['close'] - dow_jones_close['open']
dow_jones_close.head()


,timestamp,open,high,low,close,volume,close-open
0,2019-09-27,26987.2598,27012.5391,26715.8203,26820.2500,217780000,-167.0098
1,2019-09-26,27004.1094,27015.0703,26803.8398,26891.1191,229180000,-112.9903
2,2019-09-25,26866.7109,27016.5605,26755.8594,26970.7109,237220000,104.0000
3,2019-09-24,27034.0703,27079.6797,26704.9609,26807.7695,301750000,-226.3008
4,2019-09-23,26851.4492,27011.0703,26831.3398,26949.9902,204240000,98.5410


In [4]:
# This code changes the 'close-minus-open' column from a string to a float number
dow_jones_close['close-open'] = [float(x) for x in dow_jones_close['close-open']]
dow_jones_close.head()

,timestamp,open,high,low,close,volume,close-open
0,2019-09-27,26987.2598,27012.5391,26715.8203,26820.2500,217780000,-167.0098
1,2019-09-26,27004.1094,27015.0703,26803.8398,26891.1191,229180000,-112.9903
2,2019-09-25,26866.7109,27016.5605,26755.8594,26970.7109,237220000,104.0000
3,2019-09-24,27034.0703,27079.6797,26704.9609,26807.7695,301750000,-226.3008
4,2019-09-23,26851.4492,27011.0703,26831.3398,26949.9902,204240000,98.5410


In [5]:
#check that close-open are now float numbers vs. string
a = dow_jones_close['close-open'][0] - dow_jones_close['close-open'][1]
print(a)

-54.019499999998516


In [6]:
#dow_jones_close['timestamp'] = [x.replace('-', '') for x in dow_jones_close['timestamp']]
#dow_jones_close['timestamp'] = [int(x) for x in dow_jones_close['timestamp']]
#dow_jones_close.head()

In [7]:
# Remove the index column and let timestamp (which is year-month-day) be the index
dow_jones_close_no_index = dow_jones_close.set_index('timestamp')
dow_jones_close_no_index.head()


,open,high,low,close,volume,close-open
timestamp,,,,,,
2019-09-27,26987.2598,27012.5391,26715.8203,26820.2500,217780000,-167.0098
2019-09-26,27004.1094,27015.0703,26803.8398,26891.1191,229180000,-112.9903
2019-09-25,26866.7109,27016.5605,26755.8594,26970.7109,237220000,104.0000
2019-09-24,27034.0703,27079.6797,26704.9609,26807.7695,301750000,-226.3008
2019-09-23,26851.4492,27011.0703,26831.3398,26949.9902,204240000,98.5410


In [23]:
#check that close-open are now float numbers vs. string
#b = dow_jones_close['timestamp'][0] - dow_jones_close['timestamp'][1]
#print(b)

In [50]:
dow_negatives = dow_jones_close_no_index.lt(0).sum().sum() 
print(dow_negatives)

2312


In [40]:
# Total number of days the market was up (+), down (-), or neutral from 1/3/2000-9/27/2019
negatives = len(dow_jones_close_no_index.loc[(dow_jones_close_no_index['close-open'] < 0)])    
positives = len(dow_jones_close_no_index.loc[(dow_jones_close_no_index['close-open'] > 0)])
neutrals =  len(dow_jones_close_no_index.loc[(dow_jones_close_no_index['close-open'] == 0)])
print('Dow Jones negatives', negatives) 
print('Dow Jones positives', positives)
print('Dow Jones neutrals ', neutrals)


Dow Jones negatives 2312
Dow Jones positives 2651
Dow Jones neutrals  3


In [39]:
# Create a database for dates between Trump's election (11/8/2016) 
# and his inauguration (1/20/2017)    where ea stands for election to inauguration
trump_ea_df = dow_jones_close[676:726]
trump_ea_df.head()

,timestamp,open,high,low,close,volume,close-open
676,2017-01-20,19795.0605,19843.9395,19759.1406,19827.2500,435260000,32.1895
677,2017-01-19,19813.5508,19824.1406,19677.9395,19732.4004,301750000,-81.1504
678,2017-01-18,19822.7305,19828.1992,19739.0000,19804.7207,279760000,-18.0098
679,2017-01-17,19848.8203,19882.9902,19775.3496,19826.7695,341900000,-22.0508
680,2017-01-13,19912.5391,19952.0293,19849.3809,19885.7305,263780000,-26.8086


In [30]:
# Calculate the number of days the market was up (+), down (-), or neutral 
trump_ea_negatives = len(trump_ea_df.loc[(trump_ea_df['close-open'] < 0)])    
trump_ea_positives = len(trump_ea_df.loc[(trump_ea_df['close-open'] > 0)])
trump_ea_neutrals =  len(trump_ea_df.loc[(trump_ea_df['close-open'] == 0)])
print('Trump ea negatives', trump_ea_negatives) 
print('Trump ea positives', trump_ea_positives)
print('Trump ea neutrals ', trump_ea_neutrals)


Trump ea negatives 21
Trump ea positives 29
Trump ea neutrals  0


In [34]:
# Create new dataframe to hold values as we find them


In [37]:
# Create a database for dates between Trump's term to date, in other words
# his inauguration (1/20/2017) to data collection endpoint (9/27/2018)
trump_term_df = dow_jones_close[0:677]
trump_term_df.head()


,timestamp,open,high,low,close,volume,close-open
0,2019-09-27,26987.2598,27012.5391,26715.8203,26820.2500,217780000,-167.0098
1,2019-09-26,27004.1094,27015.0703,26803.8398,26891.1191,229180000,-112.9903
2,2019-09-25,26866.7109,27016.5605,26755.8594,26970.7109,237220000,104.0000
3,2019-09-24,27034.0703,27079.6797,26704.9609,26807.7695,301750000,-226.3008
4,2019-09-23,26851.4492,27011.0703,26831.3398,26949.9902,204240000,98.5410


In [38]:
# Find the number of days the market was up (+), down (-), or neutral during Trump's term
trump_term_negatives = len(trump_term_df.loc[(trump_term_df['close-open'] < 0)])    
trump_term_positives = len(trump_term_df.loc[(trump_term_df['close-open'] > 0)])
trump_term_neutrals =  len(trump_term_df.loc[(trump_term_df['close-open'] == 0)])
print('Trump term negatives', trump_term_negatives) 
print('Trump term positives', trump_term_positives)
print('Trump term neutrals ', trump_term_neutrals)

Trump term negatives 299
Trump term positives 378
Trump term neutrals  0


In [48]:
# Create a database for dates between Obama's first term election date (11/4/2008) 
# and his first term inauguration (1/20/2009)  where ea stands for election to inauguration
obama_ea1_df = dow_jones_close[2691:2743]
obama_ea1_df.head()

,timestamp,open,high,low,close,volume,close-open
2691,2009-01-20,8279.6299,8291.9805,7939.9302,7949.0898,419200000,-330.5401
2692,2009-01-16,8215.6699,8341.2002,8109.3398,8281.2197,439360000,65.5498
2693,2009-01-15,8196.2402,8286.1602,7995.1299,8212.4902,436660000,16.2500
2694,2009-01-14,8446.0098,8446.2500,8140.7202,8200.1396,355050000,-245.8702
2695,2009-01-13,8474.6104,8522.0801,8376.8799,8448.5596,304050000,-26.0508


In [57]:
# Find the number of days the market was up (+), down (-), or neutral for Obama ea 1st term
obama_ea1_negatives = len(obama_ea1_df.loc[(obama_ea1_df['close-open'] < 0)])    
obama_ea1_positives = len(obama_ea1_df.loc[(obama_ea1_df['close-open'] > 0)])
obama_ea1_neutrals =  len(obama_ea1_df.loc[(obama_ea1_df['close-open'] == 0)])
print('Obama ea1 negatives', obama_ea1_negatives) 
print('Obama ea1 positives', obama_ea1_positives)
print('Obama ea1 neutrals ', obama_ea1_neutrals)

Obama ea1 negatives 28
Obama ea1 positives 24
Obama ea1 neutrals  0


In [ ]:
# Create a database for dates of Obama's first term from inauguration to inauguration ??
#  
obama_ea1_df = dow_jones_close[2691:2743]
obama_ea1_df.head()

In [56]:
# Create a database for dates between Bush's election (11/7/2000) 
# and his inauguration (1/20/2001)    where ea stands for election to inauguration
# January 20, 2001 was a Saturday (stock market closed) so data is through 1/19/2001
bush_ea1_df = dow_jones_close[4701:4751]
bush_ea1_df.head()

,timestamp,open,high,low,close,volume,close-open
4701,2001-01-19,10686.0000,10717.7197,10564.0098,10587.5898,363100000,-98.4102
4702,2001-01-18,10584.5703,10737.2402,10567.4697,10678.2803,307320000,93.7100
4703,2001-01-17,10660.9502,10705.9297,10545.2998,10584.3398,304800000,-76.6104
4704,2001-01-16,10525.7803,10664.0498,10486.6396,10652.6602,233330000,126.8799
4705,2001-01-12,10608.7402,10623.8896,10468.0400,10525.3799,247020000,-83.3603


In [58]:
# Find the number of days the market was up (+), down (-), or neutral for Bush ea 1st term
bush_ea1_negatives = len(bush_ea1_df.loc[(bush_ea1_df['close-open'] < 0)])    
bush_ea1_positives = len(bush_ea1_df.loc[(bush_ea1_df['close-open'] > 0)])
bush_ea1_neutrals =  len(bush_ea1_df.loc[(bush_ea1_df['close-open'] == 0)])
print('Bush ea1 negatives', bush_ea1_negatives) 
print('Bush ea1 positives', bush_ea1_positives)
print('Bush ea1 neutrals ', bush_ea1_neutrals)

Bush ea1 negatives 27
Bush ea1 positives 23
Bush ea1 neutrals  0
